In [1]:
import os
import requests
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Function to fetch CPI data from FRED API
def fetch_cpi_data(api_key, start_date, end_date, series_id="CPIAUCSL"):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start_date,
        "observation_end": end_date
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        cpi_data = response.json()
        observations = cpi_data['observations']
        data = [(item['date'], float(item['value'])) for item in observations]
        df = pd.DataFrame(data, columns=['Date', 'CPI Value'])
        df['Date'] = pd.to_datetime(df['Date']) + pd.offsets.MonthEnd(0)
        return df
    else:
        raise Exception(f"Failed to retrieve data from FRED API. Status code: {response.status_code}")

# Function to load CPI data from a CSV file
def load_cpi_data_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'])
    return df

# Function to generate dummy spending data
def generate_dummy_spending_data(start_date, end_date, categories):
    dates = pd.date_range(start=start_date, end=end_date, freq='M')
    np.random.seed(42)
    data = {
        "Date": np.tile(dates, len(categories)),
        "Category": np.repeat(categories, len(dates)),
        "Spending": np.random.uniform(100, 1000, len(dates) * len(categories))
    }
    return pd.DataFrame(data)

# Function to prepare the machine learning data
def prepare_ml_data(merged_df, category):
    category_data = merged_df[merged_df['Category'] == category]
    category_data = category_data[['Date', 'Spending']].set_index('Date').resample('M').sum()

    for i in range(1, 7):
        category_data[f'Lag{i}'] = category_data['Spending'].shift(i)

    category_data = category_data.dropna()
    X = category_data[[f'Lag{i}' for i in range(1, 7)]]
    y = category_data['Spending']
    return X, y

# Function to predict the next month's CPI
def predict_next_cpi(merged_df):
    cpi_data = merged_df[['Date', 'CPI Value']].drop_duplicates().set_index('Date').resample('M').mean()
    for i in range(1, 7):
        cpi_data[f'Lag{i}'] = cpi_data['CPI Value'].shift(i)
    cpi_data = cpi_data.dropna()
    X = cpi_data[[f'Lag{i}' for i in range(1, 7)]]
    y = cpi_data['CPI Value']

    model = LinearRegression()
    model.fit(X, y)

    last_6_cpi = pd.DataFrame([X.iloc[-1].values], columns=X.columns)  # Ensure feature names are retained
    predicted_cpi = model.predict(last_6_cpi)[0]
    next_month = X.index[-1] + pd.DateOffset(months=1)
    return next_month, predicted_cpi

# Function to predict expenses and adjusted spending
def predict_expenses_and_adjusted_spending(merged_df, categories):
    predictions = []
    next_month, predicted_cpi = predict_next_cpi(merged_df)
    for category in categories:
        X, y = prepare_ml_data(merged_df, category)
        model = LinearRegression()
        model.fit(X, y)
        
        last_6_months = pd.DataFrame([X.iloc[-1].values], columns=X.columns)  # Ensure feature names are retained
        predicted_spending = model.predict(last_6_months)[0]
        adjusted_spending = predicted_spending * (predicted_cpi / merged_df['CPI Value'].iloc[0])
        predictions.append({
            "Date": next_month, 
            "Category": category, 
            "Spending": predicted_spending, 
            "CPI": predicted_cpi, 
            "Adjusted Spending": adjusted_spending
        })
    return pd.DataFrame(predictions)

# Main function to orchestrate the data fetching, processing, and Dash app setup
def main():
    api_key = "03e5ab462d33249ceadd622b5048b505"  # FRED API key
    start_date = "2024-01-01"
    end_date = "2024-07-31"
    categories = ["Groceries", "Housing", "Health", "Entertainment", "Insurance", "Transportation"]
    
    # Step 1: Fetch CPI data from FRED API and save to CSV
    try:
        cpi_df = fetch_cpi_data(api_key, start_date, end_date)
        cpi_df.to_csv('fred_cpi.csv', index=False)
        print("CPI data fetched from FRED API and saved to 'fred_cpi.csv'.")
    except Exception as e:
        print(e)
        return

    # Step 2: Load the CPI data from the CSV file
    cpi_df = load_cpi_data_from_csv('fred_cpi.csv')

    # Debugging: Check CPI values
    print("Raw CPI Data:")
    print(cpi_df.head())

    # Step 3: Generate dummy spending data
    spending_df = generate_dummy_spending_data(start_date, end_date, categories)

    # Step 4: Merge spending data with CPI data
    merged_df = pd.merge(spending_df, cpi_df, on="Date")

    # Use CPI ratio for adjustment
    base_cpi = merged_df['CPI Value'].iloc[0]
    merged_df["Adjusted Spending"] = merged_df["Spending"] * (merged_df["CPI Value"] / base_cpi)

    # Step 5: Add cumulative spending and cumulative adjusted spending
    merged_df["Cumulative Spending"] = merged_df.groupby("Category")["Spending"].cumsum()
    merged_df["Cumulative Adjusted Spending"] = merged_df.groupby("Category")["Adjusted Spending"].cumsum()

    # Step 6: Predict expenses and adjusted spending for the next month
    pred_df = predict_expenses_and_adjusted_spending(merged_df, categories)
    merged_df = pd.concat([merged_df, pred_df], ignore_index=True)
    merged_df = merged_df.sort_values(by="Date")

    # Debugging: Print merged data
    print("Merged Data:")
    print(merged_df.head())

    # Step 7: Setup and run the Dash app
    app = Dash(__name__)

    # Layout with the logo above the graph
    app.layout = html.Div([
        html.Img(src=app.get_asset_url('cash_compass.png'), style={'width': '200px', 'display': 'block', 'margin': '0 auto'}),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'By Category', 'value': 'Category'},
                {'label': 'By Month', 'value': 'Date'}
            ],
            value='Category',
            style={'width': '50%', 'margin': '20px auto'}
        ),
        dcc.Dropdown(
            id='filter-value',
            style={'width': '50%', 'margin': '10px auto'}
        ),
        dcc.Graph(id='spending-chart')
    ])

    @app.callback(
        Output('filter-value', 'options'),
        [Input('filter-type', 'value')]
    )
    def set_filter_value_options(filter_type):
        if filter_type == 'Category':
            return [{'label': cat, 'value': cat} for cat in categories]
        else:
            unique_dates = pd.to_datetime(merged_df['Date'].unique())
            return [{'label': date.strftime("%B %Y"), 'value': date} for date in unique_dates]

    @app.callback(
        Output('spending-chart', 'figure'),
        [Input('filter-type', 'value'), Input('filter-value', 'value')]
    )
    def update_graph(filter_type, filter_value):
        if filter_value is None:
            return {}

        if filter_type == 'Category':
            filtered_df = merged_df[merged_df['Category'] == filter_value]
            title = f'Spending, Adjusted Spending, and Cumulative Spending for {filter_value}'
            fig = go.Figure()

            if not filtered_df.empty:
                fig.add_trace(go.Bar(
                    x=filtered_df['Date'].dt.strftime("%B %Y"),
                    y=filtered_df['Spending'],
                    name='Spending',
                    marker_color='blue'
                ))
                fig.add_trace(go.Bar(
                    x=filtered_df['Date'].dt.strftime("%B %Y"),
                    y=filtered_df['Adjusted Spending'],
                    name='Adjusted Spending',
                    marker_color='green'
                ))
                fig.add_trace(go.Scatter(
                    x=filtered_df['Date'].dt.strftime("%B %Y"),
                    y=filtered_df['Cumulative Spending'],
                    mode='lines+markers',
                    name='Cumulative Spending',
                    marker_color='purple'
                ))
                fig.add_trace(go.Scatter(
                    x=filtered_df['Date'].dt.strftime("%B %Y"),
                    y=filtered_df['Cumulative Adjusted Spending'],
                    mode='lines+markers',
                    name='Cumulative Adjusted Spending',
                    marker_color='orange'
                ))

            fig.update_layout(barmode='group', title=title)
        
        else:  # Filter by Date
            filter_value = pd.to_datetime(filter_value)
            filtered_df = merged_df[merged_df['Date'] == filter_value]
            title = f'Spending, Adjusted Spending, and Cumulative Spending for {filter_value.strftime("%B %Y")}'
            fig = go.Figure()

            if not filtered_df.empty:
                fig.add_trace(go.Bar(
                    x=filtered_df['Category'],
                    y=filtered_df['Spending'],
                    name='Spending',
                    marker_color='blue'
                ))
                fig.add_trace(go.Bar(
                    x=filtered_df['Category'],
                    y=filtered_df['Adjusted Spending'],
                    name='Adjusted Spending',
                    marker_color='green'
                ))
                fig.add_trace(go.Scatter(
                    x=filtered_df['Category'],
                    y=filtered_df['Cumulative Spending'],
                    mode='lines+markers',
                    name='Cumulative Spending',
                    marker_color='purple'
                ))
                fig.add_trace(go.Scatter(
                    x=filtered_df['Category'],
                    y=filtered_df['Cumulative Adjusted Spending'],
                    mode='lines+markers',
                    name='Cumulative Adjusted Spending',
                    marker_color='orange'
                ))

            fig.update_layout(barmode='group', title=title)

        return fig

    app.run_server(debug=True, port=8058)

if __name__ == '__main__':
    main()


CPI data fetched from FRED API and saved to 'fred_cpi.csv'.
Raw CPI Data:
        Date  CPI Value
0 2024-01-31    309.685
1 2024-02-29    311.054
2 2024-03-31    312.230
3 2024-04-30    313.207
4 2024-05-31    313.225
Merged Data:
        Date       Category    Spending  CPI Value  Adjusted Spending  \
0 2024-01-31      Groceries  437.086107    309.685         437.086107   
1 2024-01-31        Housing  879.558531    309.685         879.558531   
2 2024-01-31         Health  263.642470    309.685         263.642470   
3 2024-01-31  Entertainment  225.544475    309.685         225.544475   
4 2024-01-31      Insurance  633.173112    309.685         633.173112   

   Cumulative Spending  Cumulative Adjusted Spending  CPI  
0           437.086107                    437.086107  NaN  
1           879.558531                    879.558531  NaN  
2           263.642470                    263.642470  NaN  
3           225.544475                    225.544475  NaN  
4           633.173112        